In [1]:
import pandas as pd
import csv
import re

import pyspark
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
import databricks.koalas as ks

from batcave.scrape_and_clean import get_amazon_list_ids, download_amazon_review_files

### Importing Amazon reviews through Spark

These are complete datasets from a project by [Julien McAuley.](http://jmcauley.ucsd.edu/data/amazon/links.html) 

In [2]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

<em>Below is a helper function to download all of the files directly to the data folder,only run if you have the space available and have not seperately downloaded the files mentioned in data/README.md</em>

In [23]:
# download_amazon_review_files()

In [3]:
movies_df = ks.read_json('data/reviews_Movies_and_TV.json')

In [ ]:
movies_df.head()

In [4]:
movies_meta = ks.read_json('data/meta_Movies_and_TV.json')

In [5]:
books_df = ks.read_json('data/reviews_Books.json')

In [6]:
books_meta = ks.read_json('data/meta_Books.json')

### Scraping Amazon for ASIN starting point
To help narrow my results, I started by scraping the top 100 DC & Marvel Graphic Novels for their ASIN, so that I could start with an understanding of where to find the reviews I want to target. 

In [10]:
#DC Comics - top ID's
dc_url_1 = "https://www.amazon.com/gp/bestsellers/books/193766/ref=pd_zg_hrsr_books"
dc_url_2 = "https://www.amazon.com/Best-Sellers-Books-DC-Comics-Graphic-Novels/zgbs/books/193766/ref=zg_bs_pg_2?_encoding=UTF8&pg=2"
dc_ids_1 = get_amazon_list_ids(dc_url_1)
dc_ids_2 = get_amazon_list_ids(dc_url_2)

In [9]:
# Marvel Comics - top ID's
marvel_url_1 = "https://www.amazon.com/gp/bestsellers/books/4400/ref=pd_zg_hrsr_books"
marvel_url_2 = "https://www.amazon.com/Best-Sellers-Books-Marvel-Comics-Graphic-Novels/zgbs/books/4400/ref=zg_bs_pg_2?_encoding=UTF8&pg=2"
marvel_ids_1 = get_amazon_list_ids(marvel_url_1)
marvel_ids_2 = get_amazon_list_ids(marvel_url_2)

In [11]:
all_ids = dc_ids_1 + dc_ids_2 + marvel_ids_1 + marvel_ids_2

Saving these id's temporarily to a csv, to preserve and make for easier refreshing of this notebook without re-scraping.

In [ ]:
with open('data/top_dc_marvel_comics.csv', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(all_ids)

# Getting the right reviews
My thought process for getting down to the reviews of those who have scored both comic books and movies is as follows:
* Find comic book ASIN's in the metadata dataset through exploration and looking at related items. Call it "all_comic_ids".
* Find any reviews for "all_comics_ids" in the book reviews dataset. This subset is "comic_reviews"
* Take reviewer ids from "comic_reviews" and find them in movie/tv reviews dataset. This subset is "movie_reviews"
* Take the reviewer ids from "movie_reviews" and filter out any missing from "comic_reviews", calling this subset "comic_reviews_both"

### 1. Finding ASINs
After reading back in list of ASINs, finding them in metadata and reviewing the data. I am loading it out to a Pandas dataframe for now as it will most likely be smaller, so faster to work with in this way:

In [7]:
with open("data/top_dc_marvel_comics.csv") as f:
    reader = csv.reader(f)
    all_ids = []
    for row in reader:
        all_ids.append(row)

In [8]:
top_comic_meta = books_meta.filter(col("asin").isin(all_ids[0])).toPandas()

In [9]:
top_comic_meta.shape

(16, 10)

In [15]:
top_comic_meta

,_corrupt_record,asin,brand,categories,description,imUrl,price,related,salesRank,title
0,None,014038572X,None,[[Books]],"According to Ponyboy, there are two kinds of p...",http://ecx.images-amazon.com/images/I/61Xc9GWp...,4.99,"([0078205409, 0440932211, 0547534264, 01403896...","(None, 3106, None, None, None, None, None, Non...",The Outsiders
1,None,0785117210,None,[[Books]],None,http://ecx.images-amazon.com/images/I/51upWJjH...,10.49,"([0785123202, 0785157050, 078512179X, 07851329...","(None, 19065, None, None, None, None, None, No...",House of M
2,None,0785121056,None,[[Books]],None,http://ecx.images-amazon.com/images/I/61iW2SXT...,11.49,"([0785131272, 0785131280, 0785166432, 07851565...","(None, 95214, None, None, None, None, None, No...",Infinity War
3,None,140122427X,None,[[Books]],None,http://ecx.images-amazon.com/images/I/81g0BQDw...,20.00,"([1401228798, 1401230970, 1401233902, 14012349...","(None, 793, None, None, None, None, None, None...",Fables: The Deluxe Edition Book One
4,None,1401229697,None,[[Books]],"Starred Review. A stunning, moving story about...",http://ecx.images-amazon.com/images/I/71%2BCEZ...,11.01,"([1935429000, 160309038X, 1603090746, 17704604...","(None, 6414, None, None, None, None, None, Non...",Daytripper
5,None,1401230067,None,[[Books]],None,http://ecx.images-amazon.com/images/I/81IQpmRp...,11.49,"([1401233023, 1401235190, 1401236901, 14012380...","(None, 11659, None, None, None, None, None, No...","John Constantine, Hellblazer, Vol. 1: Original..."
6,None,1401233791,None,[[Books]],Dennis O'Neil is the influential writer of com...,http://ecx.images-amazon.com/images/I/81hLFMDQ...,16.09,"([1401235360, 1401237215, 1401232744, 14012338...","(None, 9262, None, None, None, None, None, Non...","Batman: Knightfall, Vol. 1"
7,None,1401233384,None,[[Books]],"Geoff Johns, a Detroit native, brings a Hollyw...",http://ecx.images-amazon.com/images/I/81LZ1mW0...,10.00,"([1401234054, 1401230016, 1401234488, 14012343...","(None, 21061, None, None, None, None, None, No...",Flashpoint
8,None,1401237789,None,[[Books]],Praise for Scott Snyder'sBatman: Court of Owls...,http://ecx.images-amazon.com/images/I/91DDC3%2...,9.36,"([1401235425, 1401246028, 1401242529, 14012402...","(None, 1361, None, None, None, None, None, Non...",Batman Vol. 2: The City of Owls (The New 52)
9,None,1401242383,None,[[Books]],Warren Ellis is a prolific writer whose works ...,http://ecx.images-amazon.com/images/I/81adYxKE...,47.73,"([1401238998, 1401242758, 1616552379, 07851656...","(None, 18519, None, None, None, None, None, No...",The Planetary Omnibus


In review, see that some of the ASINs that were scraped aren't aligning to relevant material. This could generally be from those numbers being adjusted at some systematically.

In [10]:
top_meta = top_comic_meta.drop([0, 12, 13, 14, 15])

To help get more information, I used the "Related" comics feature to grab more ASINs and will use those to help determine if I can find more.

In [11]:
top_meta['related'].head()

1    ([0785123202, 0785157050, 078512179X, 07851329...
2    ([0785131272, 0785131280, 0785166432, 07851565...
3    ([1401228798, 1401230970, 1401233902, 14012349...
4    ([1935429000, 160309038X, 1603090746, 17704604...
5    ([1401233023, 1401235190, 1401236901, 14012380...
Name: related, dtype: object

Putting the 'related' ASINs in a list, then creating a new dataframe for them.

In [12]:
more_comic_ids = list(set([val for meta in top_meta.related.tolist() for val in meta[0]]))

In [13]:
more_comic_meta = books_meta.filter(col("asin").isin(more_comic_ids)).toPandas()

In [14]:
more_comic_meta['related'].head()

0    ([0375714545, 0618871713, 1570614598, 18918304...
1    ([0316107093, 0316107107, 0316107298, 03161073...
2    ([1606904388, 0345506391, 1606901605, 16069021...
3    ([1932664165, 1620101130, 1620100045, 14424659...
4    ([1560974273, 0375404538, 0375714545, 03073773...
Name: related, dtype: object

Based on reviewing what I was able to gather so far, I found a few patterns to help narrow down a subsection that contains comic books:
* A large section of comic books from the publisher Marvel Comics begin with '07851'
* A large section of comic books from the publisher DC Comics begin with '14012'
* A large section of comic books from the publisher Image Comics begin with '160706'

As these are three of the largest publishers, I am going to use an SQL query to find those in these ranges, review the data, and if it looks to fit what I expect, proceed with combining them with my previous findings into one set. 

In [15]:
query ="""
SELECT 
    asin
,   title
,   categories
FROM 
    metadata
WHERE asin LIKE """

In [16]:
image_query = query + " \'160706%\'"
books_meta.createOrReplaceTempView('metadata')
image_meta = spark.sql(image_query).toPandas()

In [17]:
image_meta.head()

,asin,title,categories
0,1607060051,Screamland,[[Books]]
1,1607060191,Ted McKeever Library Book 3: Metropol (Bk. 3),[[Books]]
2,1607060086,Bruce: The Little Blue Spruce,[[Books]]
3,1607060043,"Outlaw Territory, Vol. 1",[[Books]]
4,160706023X,Zombie Cop,[[Books]]


In [20]:
marvel_query = query + " \'07851%\'"
books_meta.createOrReplaceTempView('metadata')
marvel_meta = spark.sql(marvel_query).toPandas()

In [21]:
marvel_meta.head()

,asin,title,categories
0,0785100016,Wolverine: Killing,[[Books]]
1,0785100555,"The Punisher, a man named Frank",[[Books]]
2,078510027X,Spider-Man: Round Robin : The Sidekick's Revenge,[[Books]]
3,0785100245,Daredevil - Fall from Grace,[[Books]]
4,0785100717,The Best of Marvel 1994,[[Books]]


In [22]:
dc_query = query + " \'14012%\'"
books_meta.createOrReplaceTempView('metadata')
dc_meta = spark.sql(dc_query).toPandas()

In [23]:
dc_meta.head()

,asin,title,categories
0,1401200168,Green Lantern: Emerald Dawn II,[[Books]]
1,1401200141,"Atom, The - Archives, Volume 2 (DC Archive Edi...",[[Books]]
2,1401200338,High Roads (Cliffhanger!),[[Books]]
3,1401200346,Batman/Deathblow: After the Fire (Batman Beyon...,[[Books]]
4,1401200370,Batman: Absolution,[[Books]]


Awesome! All of those looked good in further exploration of each. With that, I combined all of the previous results with these new ones to have my working set of comic book ASINs.

In [24]:
top_merge = top_comic_meta[['asin', 'title', 'categories']]
related_merge = more_comic_meta[['asin', 'title', 'categories']]

In [25]:
all_comic_df = pd.concat([top_merge, related_merge,
                          image_meta, marvel_meta,
                          dc_meta])
all_comic_df['categories'] = all_comic_df['categories']\
                            .astype(str)
all_comic_df.head()

,asin,title,categories
0,014038572X,The Outsiders,[['Books']]
1,0785117210,House of M,[['Books']]
2,0785121056,Infinity War,[['Books']]
3,140122427X,Fables: The Deluxe Edition Book One,[['Books']]
4,1401229697,Daytripper,[['Books']]


Next, I did some additional cleaning of my comic books ASINs:
* Dropped any duplicate rows
* Found anything that was not categorized as a Book and removed it, as well as a couple more edge cases that I wanted to drop. 
* Exported this list to a csv to preserve my efforts and not have to extract from the metadata dataset any further.

In [26]:
all_comics_df = all_comic_df.drop_duplicates()

In [ ]:
all_comics_df.loc[:,'categories'] = all_comics_df.loc[:, 'categories']\
             .apply(lambda x: re.sub("\[|\]|\'", "", x))

In [28]:
all_comics_df.head()

,asin,title,categories
0,014038572X,The Outsiders,Books
1,0785117210,House of M,Books
2,0785121056,Infinity War,Books
3,140122427X,Fables: The Deluxe Edition Book One,Books
4,1401229697,Daytripper,Books


In [29]:
all_comics_df[all_comics_df['categories'] != 'Books']

,asin,title,categories
15,B00CU06XFY,None,"Books, Comics & Graphic Novels, Graphic Novels..."


In [30]:
all_comics_df = all_comics_df.drop([0,15], axis=0)

In [ ]:
all_comics_df.to_csv('data/all_comic_asin.csv')

### 2. Find comic book reviews in Books

In [31]:
comic_reviews = books_df.filter(col('asin')
                        .isin(all_comics_df.asin.tolist()))

In [32]:
print(f"Number of comic book reviews: {comic_reviews.count()}")

Number of comic book reviews: 57935


In [33]:
comic_reviews.show(1)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0316107255| [1, 1]|    4.0|PENGUIN DREAMS AN...|02 19, 2014|A3NQU1649SH0Q4|Allen Smalling "E...|Okay, but no coll...|    1392768000|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
only showing top 1 row



Saving a version of my reviews to JSON, so it is easier to work with than the larger file of all books:

In [ ]:
comic_reviews_export = comic_reviews.select(['asin', 'overall',
                                             'reviewText', 'reviewerID'])
comic_reviews_export.repartition(1).write.json('data/all_comic_reviews.json')

### 3. Find movie/tv reviews by comic book reviewers

In [34]:
comic_reviews_df = spark.read.json('data/all_comic_reviews.json')

In [35]:
good_users = list(set(comic_reviews_df.select('reviewerID').collect()))

# Ids are still stored in Spark Rows, so this removes them into a unique list
good_user_ids = [g[0] for g in good_users]

Creating a dataframe of movie/tv reviews by users who rated comics

In [36]:
mtv_reviews = movies_df.filter(col('reviewerID').isin(good_user_ids))

In [37]:
print(f"""Number of unique movie/tv reviews
        for comic book reviewers:
        {mtv_reviews.count()}""")

Number of unique movie/tv reviews
        for comic book reviewers:
        149189


<em>Taking a quick look at how many unique users I have:</em>

In [38]:
movie_and_comic_reviewers = list(
                                set(mtv_reviews
                                    .select('reviewerID').collect()
                                    )
                            )

In [39]:
print(f"""Number of users who have rated
        both comics and movies/tv: 
        {len(movie_and_comic_reviewers)}""")

Number of users who have rated
        both comics and movies/tv: 
        10366


Saving the metadata on asociated movies/tv for later model interpretation. I found a quirk my data where there are reviews that exist in the Movie/TV dataset that do not align to a ASIN in the metadata.  Since I cannot account for what they are, I am going to drop them for the time.

In [40]:
mtv_asins = list(set(mtv_reviews.select('asin').collect()))
mtv_asins = [a[0] for a in mtv_asins]
movie_reviews_meta = movies_meta.filter(col('asin').isin(mtv_asins))

In [ ]:
movie_reviews_meta.repartition(1).write.json('data/moviestv_meta')

After narrowing the metadata down to movies from the reviews, I then narrowed down the actual reviews to match this dataset and exported to preserve. As the file size is too large for upload, I had it split into 4 partitions:

In [41]:
meta_asins = list(set(movie_reviews_meta.select('asin').collect()))
meta_asins = [a[0] for a in meta_asins]
movies_w_meta = mtv_reviews.filter(col('asin').isin(meta_asins))

In [42]:
movies_w_meta.count()

103757

In [43]:
all_movie_review_data = movies_w_meta.join(movies_meta,
                                           on='asin',
                                           how='left')

In [44]:
mtv_reviews_df = all_movie_review_data.select(['asin',
                                              'overall',
                                              'title',
                                              'reviewerID',
                                              'imUrl'])

In [45]:
mtv_reviews_df.show(1)

+----------+-------+--------------------+-------------+--------------------+
|      asin|overall|               title|   reviewerID|               imUrl|
+----------+-------+--------------------+-------------+--------------------+
|0767807693|    3.0|Requiem for a Hea...|ADENUJJYKNHPO|http://ecx.images...|
+----------+-------+--------------------+-------------+--------------------+
only showing top 1 row



### 4. Filtering comic reviews by those who rated both comics & movies/tv

In [46]:
movie_reviewers = list(set(mtv_reviews_df.select('reviewerID').collect()))

movie_and_comic_reviewers = [reviewer[0] for reviewer in movie_reviewers]

In [47]:
# Quick visual check on these ids
movie_and_comic_reviewers[:5]

['A1N7CLMHL8GX47',
 'A3CDW403KI8AUF',
 'A1B9ISLYYCHIVE',
 'A1J9YT964HP4EC',
 'A2K8GFTF3W0YNU']

In [48]:
cm_reviews_df = comic_reviews_df.filter(col('reviewerID')\
                                .isin(movie_and_comic_reviewers))

Comic reviews from reviewers who also reviewed movies/tv

In [49]:
cm_reviews_df.count()

28107

In [50]:
all_comic_review_data = cm_reviews_df.join(books_meta,
                                           on='asin',
                                           how='left')

In [51]:
all_comic_review_data.count()

28107

In [ ]:
# Exporting this new set of comic reviews to JSON

mtv_reviews_df.select(['asin', 'overall', 'title','reviewerID', 'imUrl'])\
                     .repartition(1).write.json("data/movie_reviews_wtitle")

all_comic_review_data.select(['asin', 'overall', 'title','reviewerID', 'imUrl'])\
                     .repartition(1).write.json("data/comic_reviews_wtitle")

Looks like a good start! From here, I moved to the my next notebook to merge, clean, and model my data further. 